In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.globals import set_verbose, set_debug

set_debug(False)
set_verbose(False)

In [ ]:
### LLM

llm_name = "gemini-2.0-flash-lite"     # answer llm
graph_llm_name = "gemini-2.5-flash-lite" 
embed_model_name = "all-MiniLM-L6-v2"

In [4]:
# embedding model

from langchain_huggingface import HuggingFaceEmbeddings

# model_name="all-MiniLM-L6-v2"
# model_name="sentence-transformers/all-mpnet-base-v2"

embedding_model = HuggingFaceEmbeddings(model_name=embed_model_name)

/home/fuths/projects/graphrag_2/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # [does answer generation, rest of the work] llm - huggingface model

# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint


# llm = HuggingFaceEndpoint(
#     repo_id=repo_id,
#     max_new_tokens=1024,
#     do_sample=False,
#     temperature=0,
#     provider="auto",  # let Hugging Face choose the best provider for you
# )

# chat_model = ChatHuggingFace(llm=llm)

# """ usage example

# messages = "tell a joke"
# ai_msg = chat_model.invoke(messages)

# print(ai_msg.content)
# """

In [ ]:
# graph llm - cypher generation

# google gemini

from langchain_google_genai import ChatGoogleGenerativeAI

graph_llm = ChatGoogleGenerativeAI(model=graph_llm_name, temperature=0)

In [ ]:
# answering llm

# gemini
llm = ChatGoogleGenerativeAI(model=llm_name, temperature=0)

# ollama qwen3 4b
# from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="qwen3:4b", format="json", temperature=0)

/tmp/ipykernel_672463/355719928.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="qwen3:4b", format="json", temperature=0)


In [ ]:
# get and load documents

import os
from langchain_community.document_loaders import PyMuPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

pdf_folder_path = "./data_stored/"
print(f"Loading PDFs from: {pdf_folder_path}")

pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]

all_documents = []
for pdf_file in pdf_files:
    file_path = os.path.join(pdf_folder_path, pdf_file)
    print(f"  - Loading document: {pdf_file}")
    loader = PyMuPDFLoader(file_path)
    pages = loader.load()

    """ uncomment the lines below for the demo example 
    # start_index = 55  
    # end_index = 57   

    # cleaned_pages = [
    #     page for page in pages 
    #     if start_index <= page.metadata.get('page', 0) < end_index
    # ]

    # all_documents.extend(cleaned_pages)
    """

    # comment the line below for the demo example
    all_documents.extend(pages)

print(f"Loaded a total of {len(all_documents)} pages from {len(pdf_files)} PDF files.")


Loading PDFs from: ./data_stored/
  - Loading document: Essential-GraphRAG.pdf
  - Loading document: OReilly_Graph_Databases.pdf
Loaded a total of 4 pages from 2 PDF files.


In [ ]:

# Split the loaded documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=50
)
doc_splits = text_splitter.split_documents(all_documents)

print(f"Split the documents into {len(doc_splits)} chunks.")

Split the documents into 6 chunks.


In [ ]:
# creating vectordb

from langchain_milvus import Milvus

# Add to Milvus
vectorstore = Milvus.from_documents(
    documents=doc_splits,
    collection_name="rag_milvus_att_5",
    embedding=embedding_model,
    connection_args={"uri": "./milvus_ingest_att_5.db"},
)
retriever = vectorstore.as_retriever()

/home/fuths/projects/graphrag_2/.venv/lib/python3.11/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
# GraphRAG Setup
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

import time

graph = Neo4jGraph()


graph_transformer = LLMGraphTransformer(
    llm=graph_llm,
    # allowed_nodes=["Concept"],
    # node_properties=[""],
    # allowed_relationships=["AUTHORED", "", "RELATED_TO"],
)
print("1")

graph_documents = []
for i,doc in enumerate(doc_splits):
    stime = time.time()
    # Process one document at a time
    graph_doc = graph_transformer.convert_to_graph_documents([doc])
    graph_documents.extend(graph_doc)
    print(f"end of chunk, time taken: {time.time() - stime}")
    
print("2")
graph.add_graph_documents(graph_documents)
print("3")
print(f"Graph documents: {len(graph_documents)}")
print(f"Nodes from 1st graph doc:{graph_documents[0].nodes}")
print(f"Relationships from 1st graph doc:{graph_documents[0].relationships}")

/tmp/ipykernel_672463/4244973288.py:12: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


1
end of chunk, time taken: 16.40524172782898
end of chunk, time taken: 30.17276668548584
end of chunk, time taken: 23.614866733551025
end of chunk, time taken: 40.06956386566162
end of chunk, time taken: 32.98189043998718
end of chunk, time taken: 22.406184673309326
2
3
Graph documents: 6
Nodes from 1st graph doc:[Node(id='Step-Back Prompting', type='Technique', properties={}), Node(id='Llm', type='Model', properties={}), Node(id='Query Rewriting Task', type='Task', properties={}), Node(id='Natural Language Comprehension', type='Capability', properties={}), Node(id='Natural Language Generation', type='Capability', properties={}), Node(id='Authors Of The Step-Back Prompting Paper', type='Group', properties={}), Node(id='System Prompt', type='Instruction', properties={}), Node(id='Zero-Shot Prompting', type='Technique', properties={}), Node(id='Thierry Audel', type='Person', properties={}), Node(id='Few-Shot Examples', type='Concept', properties={}), Node(id='General Capabilities', type

In [12]:
# After converting to graph documents
for i, doc in enumerate(graph_documents):
    print(f"Document {i}:")
    print(f"  Nodes: {doc.nodes}")
    print(f"  Relationships: {doc.relationships}")
    print("---")

Document 0:
  Nodes: [Node(id='Step-Back Prompting', type='Technique', properties={}), Node(id='Llm', type='Model', properties={}), Node(id='Query Rewriting Task', type='Task', properties={}), Node(id='Natural Language Comprehension', type='Capability', properties={}), Node(id='Natural Language Generation', type='Capability', properties={}), Node(id='Authors Of The Step-Back Prompting Paper', type='Group', properties={}), Node(id='System Prompt', type='Instruction', properties={}), Node(id='Zero-Shot Prompting', type='Technique', properties={}), Node(id='Thierry Audel', type='Person', properties={}), Node(id='Few-Shot Examples', type='Concept', properties={}), Node(id='General Capabilities', type='Capability', properties={})]
  Relationships: [Relationship(source=Node(id='Step-Back Prompting', type='Technique', properties={}), target=Node(id='Llm', type='Model', properties={}), type='USES', properties={}), Relationship(source=Node(id='Llm', type='Model', properties={}), target=Node(id=

In [ ]:
### Retrieval Grader

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser


# prompt = PromptTemplate(
#     template="""You are a grader assessing relevance 
#     of a retrieved document to a user question. If the document contains keywords related to the user question, 
#     grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. 
    
#     Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
#     Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
     
#     Here is the retrieved document: 
#     {document}
    
#     Here is the user question: 
#     {question}
#     """,
#     input_variables=["question", "document"],
# )

prompt = PromptTemplate(
    template="""You are a grader assessing relevance
of a retrieved document to a user question. If the document contains keywords related to the user question,
grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals.

Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
Provide the binary score as a JSON with a single key 'score' and no premable or explaination.

IMPORTANT: Your response MUST be only the JSON object itself, without any surrounding text or markdown.

Here is the retrieved document:
{document}

Here is the user question:
{question}
""",
    input_variables=["question", "document"],
)


retrieval_grader = prompt | llm | JsonOutputParser()
question = "what is Step-back prompting"
docs = retriever.invoke(question)
print(len(docs))
print(docs,'\n')
doc_txt = docs[1].page_content
print(
    f'Is our answer relevant to the question asked: {retrieval_grader.invoke({"question": question, "document": doc_txt})}'
)

In [ ]:
### Generate

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use six sentences maximum and keep the answer concise:
    Question: {question} 
    Context: {context} 
    Answer: 
    """,
    input_variables=["question", "document"],
)



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = prompt | llm | StrOutputParser()

question = "what is Step-back prompting"
docs = retriever.invoke(question)
print(len(docs))
print(docs,'\n')
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

4
[Document(metadata={'author': 'Tomaž Bratanic and Oskar Hane', 'creationDate': 'D:20250707132929Z', 'creationdate': '2025-07-07T13:29:29+00:00', 'creator': 'FrameMaker 16.0.1', 'file_path': './data_stored/Essential-GraphRAG.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': "D:20250714121407+02'00'", 'moddate': '2025-07-14T12:14:07+02:00', 'page': 55, 'pk': 461791213351337984, 'producer': 'Acrobat Distiller 20.0 (Windows)', 'source': './data_stored/Essential-GraphRAG.pdf', 'subject': '', 'title': 'Essential GraphRAG', 'total_pages': 179, 'trapped': ''}, page_content='35\n3.1\nStep-back prompting\nrange of information, making it easier for the model to identify relevant facts without\ngetting bogged down by the specifics.\n The authors used an LLM for the query rewriting task, as shown in figure 3.3.\nLLMs are an excellent fit for query-rewriting tasks as they excel at natural language\ncomprehension and generation. You don’t have to train or finetune a new model for\neach task. In

In [ ]:
### Graph Generate

from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI


cypher_prompt = PromptTemplate(
    template="""You are an expert at generating Cypher queries for Neo4j.
    Use the following schema to generate a Cypher query that answers the given question.
    Make the query flexible by using case-insensitive matching and partial string matching where appropriate.
    
    
    Schema:
    {schema}
    
    Question: {question}
    
    Cypher Query:""",
    input_variables=["schema", "question"],
)


qa_prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    Use the following Cypher query results to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise. If topic information is not available, just say that topic information is not available.
    
    Question: {question} 
    Cypher Query: {query}
    Query Results: {context} 
    
    Answer:""",
    input_variables=["question", "query", "context"],
)


graph_rag_chain = GraphCypherQAChain.from_llm(
    cypher_llm=graph_llm,
    qa_llm=llm,
    validate_cypher=True,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
    return_direct=True,
    cypher_prompt=cypher_prompt,
    qa_prompt=qa_prompt,
    allow_dangerous_requests=True
)

question = "What is Step-back prompting"
generation = graph_rag_chain.invoke({"query": question})
print(generation)



> Entering new GraphCypherQAChain chain...


KeyboardInterrupt: 

In [ ]:
### Composite Vector + Graph Generations

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.base import Chain


prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context from a vector store and a graph database to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise:
    Question: {question} 
    Vector Context: {context} 
    Graph Context: {graph_context}
    Answer: 
    """,
    input_variables=["question", "context", "graph_context"],
)


# Example input data
question = "what is Step-back prompting"

In [ ]:
# Get vector + graph answers
docs = retriever.invoke(question)

print(len(docs))
print(docs,'\n')

[Document(metadata={'author': 'Tomaž Bratanic and Oskar Hane', 'creationDate': 'D:20250707132929Z', 'creationdate': '2025-07-07T13:29:29+00:00', 'creator': 'FrameMaker 16.0.1', 'file_path': './data_stored/Essential-GraphRAG.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': "D:20250714121407+02'00'", 'moddate': '2025-07-14T12:14:07+02:00', 'page': 55, 'pk': 461791213351337984, 'producer': 'Acrobat Distiller 20.0 (Windows)', 'source': './data_stored/Essential-GraphRAG.pdf', 'subject': '', 'title': 'Essential GraphRAG', 'total_pages': 179, 'trapped': ''}, page_content='35\n3.1\nStep-back prompting\nrange of information, making it easier for the model to identify relevant facts without\ngetting bogged down by the specifics.\n The authors used an LLM for the query rewriting task, as shown in figure 3.3.\nLLMs are an excellent fit for query-rewriting tasks as they excel at natural language\ncomprehension and generation. You don’t have to train or finetune a new model for\neach task. Inst

In [ ]:
vector_context = rag_chain.invoke({"context": docs, "question": question})

print(vector_context)

Step-back prompting rewrites a question into a more generic one. This makes it easier for the model to identify relevant facts. It uses an LLM to transform a specific query into a broader question.


In [ ]:
graph_context = graph_rag_chain.invoke({"query": question})

print(graph_context)



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `abstract` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=4, column=19, offset=104>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 104, 'line': 4, 'column': 19}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "cypher\nMATCH (p:Paper)\nWHERE TOLOWER(p.title) CONTAINS TOLOWER('Step-back prompting')\nRETURN p.title, p.abstract, p.url\n"
Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `url` does not exist in database `neo4j`.

Generated Cypher:
cypher
MATCH (p:Paper)
WHERE TOLOWER(p.title) CONTAINS TOLOWER('Step-back prompting')
RETURN p.title, p.abstract, p.url


> Finished chain.
{'query': 'what is Step-back prompting', 'result': [], 'intermediate_steps': [{'query': "cypher\nMATCH (p:Paper)\nWHERE TOLOWER(p.title) CONTAINS TOLOWER('Step-back prompting')\nRETURN p.title, p.abstract, p.url\n"}]}


In [ ]:
composite_chain = prompt | llm | StrOutputParser()
answer = composite_chain.invoke(
    {"question": question, "context": vector_context, "graph_context": graph_context}
)

print(answer)

Step-back prompting rewrites a question into a more generic one. This helps the model identify relevant facts. It uses an LLM to transform a specific query into a broader question.


In [ ]:
### Hallucination Grader


prompt = PromptTemplate(
    template="""You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation.
    
    Here are the facts:
    {documents} 

    Here is the answer: 
    {generation}
    """,
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [ ]:
### Answer Grader


prompt = PromptTemplate(
    template="""You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     
    Here is the answer:
    {generation} 

    Here is the question: {question}
    """,
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'no'}

In [ ]:
### Router

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser


prompt = PromptTemplate(
    template="""You are an expert at routing a user question to the most appropriate data source. 
    You have three options:
    1. 'vectorstore': Use for questions about explanations of topics.
    2. 'graphrag': Use for questions that involve relationships between concepts.
    3. 'web_search': Use for all other questions or when current information is needed.

    You do not need to be stringent with the keywords in the question related to these topics. 
    Choose the most appropriate option based on the nature of the question.

    Return a JSON with a single key 'datasource' and no preamble or explanation. 
    The value should be one of: 'vectorstore', 'graphrag', or 'web_search'.
    
    Question to route: 
    {question}""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()
question = "what is Step-back prompting"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

/tmp/ipykernel_672463/3223396883.py:28: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


{'datasource': 'vectorstore'}


In [ ]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

/tmp/ipykernel_672463/730760015.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(k=3)


We'll implement these as a control flow in LangGraph.

In [ ]:
from typing_extensions import TypedDict
from typing import List

### State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
        graph_context: results from graph search
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]
    graph_context: str


from langchain.schema import Document

### Nodes
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer using RAG on retrieved documents and graph context

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state.get("documents", [])
    graph_context = state.get("graph_context", "")

    # Composite RAG generation
    generation = composite_chain.invoke(
        {"question": question, "context": documents, "graph_context": graph_context}
    )
    return {
        "documents": documents,
        "question": question,
        "generation": generation,
        "graph_context": graph_context,
    }


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


def web_search(state):
    """
    Web search based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])  # Use get() with a default empty list

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}


### Conditional edge
def route_question(state):
    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})
    print(source)
    print(source["datasource"])

    if source["datasource"] == "graphrag":
        print("---TRYING GRAPH SEARCH---")
        graph_result = graph_search({"question": question})
        if graph_result["graph_context"] != "No results found in the graph database.":
            return "graphrag"
        else:
            print("---NO RESULTS IN GRAPH, FALLING BACK TO VECTORSTORE---")
            return "retrieve"
    elif source["datasource"] == "vectorstore":
        print("---ROUTE QUESTION TO VECTORSTORE RAG---")
        return "retrieve"
    elif source["datasource"] == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def graph_search(state):
    """
    Perform GraphRAG search using Neo4j

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updated state with graph search results
    """
    print("---GRAPH SEARCH---")
    question = state["question"]

    # Use the graph_rag_chain to perform the search
    result = graph_rag_chain.invoke({"query": question})

    # Extract the relevant information from the result
    # Adjust this based on what graph_rag_chain returns
    graph_context = result.get("result", "")

    # You might want to combine this with existing documents or keep it separate
    return {"graph_context": graph_context, "question": question}


### Conditional edge
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = grade = score.get("score", "").lower()

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"


from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("graphrag", graph_search)

### Graph Build

In [ ]:
# Set conditional entry point
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "retrieve": "retrieve",
        "graphrag": "graphrag",
    },
)

# Add edges
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("graphrag", "generate")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [ ]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint

inputs = {"question": "what is Step-back prompting?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
what is Step-back prompting?
{'datasource': 'vectorstore'}
vectorstore
---ROUTE QUESTION TO VECTORSTORE RAG---
---RETRIEVE---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finished running: grade_documents:'
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('Step-back prompting is a technique that encourages a language model to '
 'consider abstractions and first principles before answering a question. It '
 'involves two steps: abstraction, where the model identifies higher-level '
 'concepts, and reason

In [ ]:
# # Compile
# app = workflow.compile()

# # Test
# from pprint import pprint

# inputs = {"question": "Did Emmanuel Macron visit Germany recently?"}
# for output in app.stream(inputs):
#     for key, value in output.items():
#         pprint(f"Finished running: {key}:")
# pprint(value["generation"])

In [ ]:
# # Test
# from pprint import pprint

# inputs = {"question": "Which paper talk about large language models?"}
# for output in app.stream(inputs):
#     for key, value in output.items():
#         pprint(f"Finished running: {key}:")
# pprint(value["generation"])